# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [3]:
!pip install selenium

     ---------------------------------------- 6.3/6.3 MB 3.2 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [30]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
navegador = webdriver.Chrome()

#----- Para rodar o chrome em segundo plano ------------------- 
#from selenium.webdriver.chrome.options import Options
#chrome_options = Options()
#chrome_options.headless = True
#navegador = webdriver.Chrome(options=chrome_options)

navegador.get("https://www.google.com/")

busca = navegador.find_element('xpath', '//input[@title="Pesquisar"]')
busca.send_keys('Cotação dolar')
busca.send_keys(Keys.ENTER) #MWvIVe egcvbb

inputDolar = navegador.find_element('xpath', "//*[@data-name='Real brasileiro']//input[@aria-label='Campo do montante da moeda']")
cotacaoDolar = inputDolar.get_attribute('value')

navegador.get("https://www.google.com/")

busca = navegador.find_element('xpath', '//input[@title="Pesquisar"]')
busca.send_keys('Cotação euro')
busca.send_keys(Keys.ENTER) #MWvIVe egcvbb

inputEuro = navegador.find_element('xpath', "//*[@data-name='Real brasileiro']//input[@aria-label='Campo do montante da moeda']")
cotacaoEuro = inputEuro.get_attribute('value')

navegador.get("https://www.melhorcambio.com/ouro-hoje")

labelCotacaoOuro = navegador.find_element('xpath', "//*[@id='comercial']")
cotacaoOuro = labelCotacaoOuro.get_attribute('value').replace(',', '.')

navegador.quit()

In [35]:
print(cotacaoDolar)
print(cotacaoEuro)
print(cotacaoOuro)

5.18
5.61
321.72


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [34]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [38]:
#atualizar cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacaoDolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacaoEuro )
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacaoOuro )

#atualizar os preços
#recalcular os preços 
tabela["Preço de Compra"] = tabela["Cotação"] * tabela["Preço Original"]

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.18,5179.9482,1.40,7251.92748
1,Carro Renault,4500.00,Euro,5.61,25245.0000,2.00,50490.00000
2,Notebook Dell,899.99,Dólar,5.18,4661.9482,1.70,7925.31194
3,IPhone,799.00,Dólar,5.18,4138.8200,1.70,7035.99400
4,Carro Fiat,3000.00,Euro,5.61,16830.0000,1.90,31977.00000
5,Celular Xiaomi,480.48,Dólar,5.18,2488.8864,2.00,4977.77280
6,Joia 20g,20.00,Ouro,321.72,6434.4000,1.15,7399.56000


### Agora vamos exportar a nova base de preços atualizada

In [39]:
tabela.to_excel("Produtos.xlsx", index=False)